In [1]:
!nvidia-smi

Thu Dec 17 15:49:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 41%   59C    P0    98W / 280W |     18MiB / 24219MiB |     11%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 40%   41C    P8     4W / 280W |      1MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
import os

import torch
import random
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets
from torchvision import transforms

import torch
import torch.nn as nn
import torch.optim as optim

import torch.utils.data
from PIL import Image

from time import time

In [3]:
#! unzip 00000.zip 
#! unzip "01000.zip"
#! unzip "without_mask.zip"
#! unzip "with_mask.zip"

In [4]:
#! mv "./00000/" "./corr"
#! mv "./01000/" "./incorr"
#!rm ./without_mask/.DS_Store
#!cp -a ./with_mask/. ./corr/
#!rm ./corr/.DS_Store

In [5]:
'''
import shutil

with open(outfilename, 'wb') as outfile:
    for filename in glob.glob('*.txt'):
        if filename == outfilename:
            # don't want to copy the output into the output
            continue
        with open(filename, 'rb') as readfile:
            shutil.copyfileobj(readfile, outfile)
'''

"\nimport shutil\n\nwith open(outfilename, 'wb') as outfile:\n    for filename in glob.glob('*.txt'):\n        if filename == outfilename:\n            # don't want to copy the output into the output\n            continue\n        with open(filename, 'rb') as readfile:\n            shutil.copyfileobj(readfile, outfile)\n"

In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs_corr = list(sorted(os.listdir(os.path.join(root, "corr"))))
        self.imgs_incor = list(sorted(os.listdir(os.path.join(root, "incorr"))))
        self.no_mask = list(sorted(os.listdir(os.path.join(root, "without_mask"))))

    def __getitem__(self, idx):
        # load images ad masks
        if idx < len(self.imgs_corr):
            img_path = os.path.join(self.root, "corr", self.imgs_corr[idx])
            target = 1
        elif idx < len(self.imgs_corr) + len(self.imgs_incor):
            img_path = os.path.join(self.root, "incorr", self.imgs_incor[idx - len(self.imgs_corr)])
            target = 0
        else:
            img_path = os.path.join(self.root, "without_mask", self.no_mask[idx - len(self.imgs_corr) -\
                                                                                  len(self.imgs_incor)])
            target = 2
        
        img = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs_corr) + len(self.imgs_incor) + len(self.no_mask)

In [7]:
dataset = CustomDataset('./')
img, t = dataset[300]

In [8]:
len(dataset)

5740

In [9]:
input_size = 224

In [10]:
train_tf = transforms.Compose([
    transforms.Resize(input_size),
    transforms.RandomCrop((input_size, input_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(7),
    transforms.ToTensor()
])

#тест не меняем
test_tf = transforms.Compose([
    transforms.Resize(input_size),
    transforms.RandomCrop((input_size, input_size)),
    transforms.ToTensor()
])

In [11]:
test_size = 300

In [12]:
# use our dataset and defined transformations
dataset = CustomDataset('./', train_tf)
dataset_test = CustomDataset('./', test_tf)

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-300])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-300:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=16, shuffle=True)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=8, shuffle=False)

In [13]:
len(dataset)

5440

In [14]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [15]:
## specify the GPU id's, GPU id's start from 0.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device we will work on:', device)

Device we will work on: cuda:0


densenet161

In [16]:
import torchvision.models as models
model = models.densenet161(pretrained=True)

In [17]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [18]:
#В качестве лоса возмем кросс-энтропию. Оптимизатор - Адам
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

#добавим уменьшение лернинг рейта, если выходим на плато. Это решение будем принимать по валидационной выборке.
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=7, factor=0.2)

In [19]:
num_classes=3

In [20]:
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, num_classes)
input_size = 224

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 26,478,627 trainable parameters


In [22]:
model = model.to(device)

fine-tuning

In [23]:
def train(epoch, trainloader, testloader):
    running_loss = 0.0
    train_acc = 0.0
    test_acc = 0.0
    len_train = 0.0
    len_test = 0.0
    
    loss_train = 0.0
  
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss_val = loss(outputs, targets)
        loss_val.backward()
        optimizer.step()
        accuracy_add = (outputs.argmax(dim=1) == targets).float().sum().data.cpu()
        train_acc += accuracy_add
        len_train += len(targets)
        loss_train += len(targets) * loss_val.item()
        running_loss += loss_val.item()
        
    for _, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        accuracy_add = (outputs.argmax(dim=1) == targets).float().sum().data.cpu()
        test_acc += accuracy_add
        len_test += len(targets)

        
    lr_scheduler.step(running_loss)

    return train_acc / len_train, test_acc / len_test

In [24]:
accuracy_history_test = []
accuracy_history_train = []

n_epochs = 10

for epoch in range(n_epochs):
    start_time = time() 
    train_acc, test_acc = train(epoch, data_loader, data_loader_test)
    accuracy_history_test.append(test_acc)
    accuracy_history_train.append(train_acc)
    acc_train = np.round(train_acc.numpy(), 4)
    acc_test = np.round(test_acc.numpy (), 4)
    print('Epoch:', epoch+1, '   acc_train:', acc_train, '   test_acc:', acc_test, '   time: %.2f'%(time() - start_time))

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch: 1    acc_train: 0.9233    test_acc: 0.93    time: 284.05
Epoch: 2    acc_train: 0.9612    test_acc: 0.9267    time: 225.41
Epoch: 3    acc_train: 0.9744    test_acc: 0.9667    time: 230.77
Epoch: 4    acc_train: 0.9763    test_acc: 0.9633    time: 247.81
Epoch: 5    acc_train: 0.9824    test_acc: 0.9767    time: 231.25
Epoch: 6    acc_train: 0.9855    test_acc: 0.9733    time: 235.38
Epoch: 7    acc_train: 0.9866    test_acc: 0.9833    time: 227.35
Epoch: 8    acc_train: 0.9807    test_acc: 0.9767    time: 251.89
Epoch: 9    acc_train: 0.9858    test_acc: 0.97    time: 266.45
Epoch: 10    acc_train: 0.9866    test_acc: 0.98    time: 242.53


In [25]:
model.eval()
torch.save(model, 'dense161.pth')